In [1]:
import pandas as pd
import numpy as np
## plotting libraries
from matplotlib import pyplot as plt
## Sklearn libraries
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("DATA_Customer-Churn.csv")

In [5]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].apply(lambda x: x.strip()))

In [6]:
#df.isnull().any()
df = df.dropna()
df.isna().value_counts()

gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport  StreamingTV  StreamingMovies  Contract  MonthlyCharges  TotalCharges  Churn
False   False          False    False       False   False         False           False         False             False        False        False            False     False           False         False    7032
dtype: int64

In [7]:
df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)

In [8]:
cols_to_keep = ["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges", "Churn"]
df2 = df[cols_to_keep]

# split the dataset

In [32]:
df["Churn"].value_counts()

No     5163
Yes    1869
Name: Churn, dtype: int64

In [9]:
# X-y split
X = df2[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]
y = df2["Churn"].map({"Yes" : 1, "No" : 0})

In [10]:
# define functions for splitting
def split_the_shit(X, y):
    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)
    # select numerical columns
    X_train = pd.DataFrame(X_train, columns=X.columns)
    X_test  = pd.DataFrame(X_test, columns=X.columns)
    return X_train, X_test, y_train, y_test

# ...and scaling
def scale_fct(X_train):
    # now transform with StandardScaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_scaled = scaler.transform(X_train)
    return pd.DataFrame(X_scaled, columns=X_train.columns)

In [11]:
# apply test-train split
X_train, X_test, y_train, y_test = split_the_shit(X, y)

In [12]:
# apply standard scaler
X_scaled = scale_fct(X_train)

# fit and crossvalidate KNeighboursClassifier model

In [13]:
model = KNeighborsClassifier(n_neighbors=2,weights='uniform') # declare we're using knn classification model
model.fit(X_train, y_train) # train model
y_pred = model.predict(X_test) # predict test
y_pred_train = model.predict(X_train) # predict train (for sanity checks)
#model.predict_proba(inputdata)

In [19]:
# cross validate KNeighboursClassifier
scores = cross_val_score(model, X_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.77777778 0.77333333 0.768      0.75911111 0.76177778]
Score stats: 0.77 accuracy with a standard deviation of 0.01


# fit and crossvalidate logistic regression model

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix

log_model = LogisticRegression()

# Data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)

trans = PowerTransformer()

trans.fit(X_train)

X_train_mod = trans.transform(X_train)
X_test_mod  = trans.transform(X_test)

log_model.fit(X_train_mod, y_train)

y_pred_train_log = log_model.predict(X_train_mod)
y_pred_test_log = log_model.predict(X_test_mod)

performance_log = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, y_pred_train_log),
                                         precision_score(y_train, y_pred_train_log),
                                         recall_score(y_train, y_pred_train_log)],
                               'Test': [accuracy_score(y_test, y_pred_test_log),
                                        precision_score(y_test, y_pred_test_log),
                                        recall_score(y_test, y_pred_test_log)]})

display(performance_log)


,Error_metric,Train,Test
0,Accuracy,0.794311,0.781805
1,Precision,0.658266,0.675676
2,Recall,0.443916,0.439698


In [22]:
# cross validate KNeighboursClassifier
scores = cross_val_score(log_model, X_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.79111111 0.79822222 0.77511111 0.80444444 0.80888889]
Score stats: 0.80 accuracy with a standard deviation of 0.01


# fit and crossvalidate logistic regression model

In [23]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)

# Bear in mind that sklearn uses a different function for decission trees used for
# classification ( to predict a categorical feature ): DecisionTreeClassifier()
model = DecisionTreeClassifier(max_depth=6)

model.fit(X_train, y_train)

y_pred_train_dt = model.predict(X_train)
y_pred_test_dt = model.predict(X_test)


performance_df = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, y_pred_train_dt),
                                         precision_score(y_train, y_pred_train_dt),
                                         recall_score(y_train, y_pred_train_dt)],
                               'Test': [accuracy_score(y_test, y_pred_test_dt),
                                        precision_score(y_test, y_pred_test_dt),
                                        recall_score(y_test, y_pred_test_dt)]})

display(performance_df)

,Error_metric,Train,Test
0,Accuracy,0.811200,0.757640
1,Precision,0.693472,0.604396
2,Recall,0.498300,0.414573


In [24]:
# cross validate KNeighboursClassifier
scores = cross_val_score(log_model, X_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.79111111 0.79822222 0.77511111 0.80444444 0.80888889]
Score stats: 0.80 accuracy with a standard deviation of 0.01


# all in one

In [36]:
model1 = KNeighborsClassifier(n_neighbors=5,weights='uniform')
model2 = LogisticRegression()
model3 = DecisionTreeClassifier(max_depth=6)

model_pipeline = [model1, model2, model3]
model_names = ['KNeighbours', 'Logistic Regression', 'Decision Tree']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
scores_df = pd.DataFrame([scores], columns=model_names)

In [37]:
scores_df

,KNeighbours,Logistic Regression,Decision Tree
0,0.778489,0.795556,0.783467


# check for imbalances

In [41]:
df.groupby("Churn").count()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges
Churn,,,,,,,,,,,,,,,
No,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163
Yes,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869


In [71]:
# create and clean new dataset
def load_clean_df():
    df = pd.read_csv("DATA_Customer-Churn.csv")
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].apply(lambda x: x.strip()))
    df = df.dropna()
    df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)
    cols_to_keep = ["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges", "Churn"]
    df2 = df[cols_to_keep]
    return df2

In [89]:
# use SMOTE to upsample the Churn == Yes datapoints in order to balance the dataset
from imblearn.over_sampling import SMOTE

smote = SMOTE()
# refresh the data and do X,y split
data = load_clean_df()

print(data.shape)

y = data['Churn']
X = data.drop(['Churn'], axis=1)

# now do the test-train split
X_train, X_test, y_train, y_test = split_the_shit(X, y)

print(X_train.shape)

# now do the upsampling
X_sm, y_sm = smote.fit_resample(X_train, y_train)
#y_sm.value_counts()

(7032, 5)
(5625, 4)


In [112]:
from sklearn.metrics import precision_score, recall_score

model1 = KNeighborsClassifier(n_neighbors=2,weights='uniform')
model2 = LogisticRegression()
model3 = DecisionTreeClassifier(max_depth=6)

model_pipeline = [model1, model2, model3]
model_names = ['KNeighbours', 'Logistic Regression', 'Decision Tree']
scores = {}

i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
scores_df = pd.DataFrame([scores], columns=model_names)

In [ ]:
from sklearn.metrics import make_scorer, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.svm import SVC

# define the models
model1 = KNeighborsClassifier(n_neighbors=2,weights='uniform')
model2 = LogisticRegression()
model3 = DecisionTreeClassifier(max_depth=6)

# define the scoring metrics
scoring = {'accuracy': 'accuracy',
           'precision': make_scorer(precision_score, pos_label='Yes'),
           'recall': make_scorer(recall_score, pos_label='Yes')}

# define the cross-validation method
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# perform cross-validation and get the scores
scores_model1 = cross_validate(model1, X, y, cv=skf, scoring=scoring)
scores_model2 = cross_validate(model2, X, y, cv=skf, scoring=scoring)
scores_model3 = cross_validate(model3, X, y, cv=skf, scoring=scoring)

# combine the scores into a dataframe
df_scores = pd.DataFrame({'Model 1 Accuracy': scores_model1['test_accuracy'],
                          'Model 1 Precision': scores_model1['test_precision'],
                          'Model 1 Recall': scores_model1['test_recall'],
                          'Model 2 Accuracy': scores_model2['test_accuracy'],
                          'Model 2 Precision': scores_model2['test_precision'],
                          'Model 2 Recall': scores_model2['test_recall'],
                          'Model 3 Accuracy': scores_model3['test_accuracy'],
                          'Model 3 Precision': scores_model3['test_precision'],
                          'Model 3 Recall': scores_model3['test_recall']})

In [ ]:
df_scores

In [107]:
# not working yet
# how to get precision and recall - have to do manually and can't be done on estimator level
# therefore, this needs to be put inside the loop of the previous cell
from sklearn import svm


# not sure what exactly these do:
# These lines of code are related to machine learning using the scikit-learn library in Python. Specifically, they are used for performing cross-validation of a Support Vector Machine (SVM) model with a linear kernel.
# StratifiedKFold(n_splits=10) initializes a cross-validation generator that splits the data into 10 folds, preserving the percentage of samples for each class. This means that each fold will have roughly the same proportion of samples from each class as the complete dataset.
# svm.SVC(kernel='linear', C=1) initializes an SVM classifier with a linear kernel and a regularization parameter C of 1. The linear kernel is used to map the input data to a higher-dimensional space where a linear separation is possible. The regularization parameter C controls the tradeoff between maximizing the margin between classes and minimizing the classification error on the training data.

skf = StratifiedKFold(n_splits=5)
clf = svm.SVC(kernel='linear', C=1)

precision_scores = []
recall_scores = []
# integrate into for loop above to iterate over all models
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    y_pred = model1.fit(X_train, y_train).predict(X_test)
    precision_scores.append(precision_score(y_test, y_pred, average='micro'))
    recall_scores.append(recall_score(y_test, y_pred, average='micro'))

print(precision_scores)
print("Recall: %0.2f (+/- %0.2f)" % (np.mean(precision_scores), np.std(precision_scores) * 2))
print(recall_scores)
print("Recall: %0.2f (+/- %0.2f)" % (np.mean(recall_scores), np.std(recall_scores) * 2))

SyntaxError: invalid syntax (3362167730.py, line 10)